In [7]:
from kafka import KafkaProducer
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import pandas as pd
import json

import requests
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

In [8]:
def get_source(url):
    """Return the source code for the provided URL. 

    Args: 
        url (string): URL of the page to scrape.

    Returns:
        response (object): HTTP response object from requests_html. 
    """

    try:
        session = HTMLSession()
        response = session.get(url)
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [9]:
def get_feed(url):
    """Return a Pandas dataframe containing the RSS feed contents.

    Args: 
        url (string): URL of the RSS feed to read.

    Returns:
        df (dataframe): Pandas dataframe containing the RSS feed contents.
    """
    
    response = get_source(url)
    
    df = pd.DataFrame(columns = ['title', 'pubDate', 'link', 'description'])

    with response as r:
        items = r.html.find("item", first=False)

        for item in items:        

            title = item.find('title', first=True).text
            pubDate = item.find('pubDate', first=True).text
            link = item.find('guid', first=True).text
            description = item.find('description', first=True).text


            row = {'title': title, 'pubDate': pubDate, 'link': link, 'description': description}
            df = df.append(row, ignore_index=True)

    return df

In [23]:
#url='https://www.01net.com/actualites/feed/'
#url='https://www.01net.com/actualites/feed/'
url='https://blog.ovhcloud.com/feed/'

In [24]:
def ouvrir_fenetre():
    sites = {
        "Le Monde": "https://www.lemonde.fr/rss/une.xml",
        "OVH Cloud": "https://blog.ovhcloud.com/feed/",
        "Twitter": "https://www.twitter.com",
        "Reddit": "https://www.reddit.com"
    }

    def ajouter_site():
        selection.clear()
        for site in listebox.curselection():
            nom_site = listebox.get(site)
            selection.append(sites[nom_site])
        fenetre.quit()

    fenetre = tk.Tk()

    listebox = tk.Listbox(fenetre, selectmode=tk.MULTIPLE)
    for site in sites:
        listebox.insert(tk.END, site)

    bouton_ajouter = tk.Button(fenetre, text="Ajouter", command=ajouter_site)

    selection = []

    listebox.grid(row=0, column=0)
    bouton_ajouter.grid(row=1, column=0)

    fenetre.mainloop()

    return selection

In [26]:
data=get_feed(url)

/Users/antoinesusini/opt/anaconda3/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(
/var/folders/kb/kq8rbcl93hq1t3bmnp4b0yj00000gn/T/ipykernel_43284/1732422673.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(row, ignore_index=True)
/var/folders/kb/kq8rbcl93hq1t3bmnp4b0yj00000gn/T/ipykernel_43284/1732422673.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat ins

In [27]:
data

,title,pubDate,link,description
0,Benchmarking Prometheus like a pro with k6,"Tue, 04 Apr 2023 12:19:05 +0000",https://blog.ovhcloud.com/?p=24585,<![CDATA[\nIn our previous posts about choosin...
1,Understanding OVHcloud’s data centres: how Del...,"Tue, 21 Mar 2023 09:40:00 +0000",https://blog.ovhcloud.com/?p=24781,<![CDATA[\nIn our latest installment we introd...
2,Benchmarking Prometheus promql performance,"Fri, 17 Mar 2023 12:00:00 +0000",https://blog.ovhcloud.com/?p=24598,"<![CDATA[\nHere @OVHCloud, we try to replace o..."
3,Prometheus’ remote storage playground,"Sun, 05 Mar 2023 23:49:35 +0000",https://blog.ovhcloud.com/?p=24583,<![CDATA[\nIntroduction In the previous post w...
4,Space: What lies above the Cloud – Part II,"Wed, 01 Mar 2023 14:45:00 +0000",https://blog.ovhcloud.com/?p=24812,<![CDATA[\nAs we mentioned in our previous art...
5,Welcome to Prometheus world of remote storage,"Thu, 16 Feb 2023 16:29:25 +0000",https://blog.ovhcloud.com/?p=24579,"<![CDATA[\nAt OVHcloud, we recently made a cha..."
6,Transforming networks into business enablers w...,"Mon, 13 Feb 2023 17:04:44 +0000",https://blog.ovhcloud.com/?p=24673,<![CDATA[\nWhy trust in networks is essential ...
7,Transformer les réseaux en catalyseurs d’activ...,"Mon, 13 Feb 2023 17:04:21 +0000",https://blog.ovhcloud.com/?p=24672,<![CDATA[\nPourquoi la confiance dans les rése...
8,Why sustainability should be a priority for st...,"Fri, 10 Feb 2023 11:39:49 +0000",https://blog.ovhcloud.com/?p=24638,<![CDATA[\nSustainability has become an increa...
9,Ransomware targeting VMware ESXi,"Fri, 03 Feb 2023 16:10:29 +0000",https://blog.ovhcloud.com/?p=24513,<![CDATA[\nA wave of attacks is currently targ...


In [21]:
#data['title']=data['title'].str.extract(r'.*\[(.*?)\].*',expand=False)
#data['description']=data['description'].str.extract(r'.*\[(.*?)\].*',expand=False)

# extraire le texte entre les balises "[" et "]" uniquement si les balises sont présentes
data['title'] = data['title'].apply(lambda x: x.replace('ê', 'e'))
for i in range(len(data)):
    data['description'] = data['description'].apply(lambda x: x.replace('ê', 'e'))
    if '[' in data.loc[i,'title'] and ']' in data.loc[i,'title']:
        data.loc[i,'title'] = data.loc[i,'title'].split('<![CDATA[')[1].split(']')[0] 
    if '[' in data.loc[i,'description'] and ']' in data.loc[i,'description']:
        data.loc[i,'description'] = data.loc[i,'description'].split('<![CDATA[')[1].split(']')[0] 
    else:
        data.loc[i,'title'] = data.loc[i,'title']
        data.loc[i,'description'] = data.loc[i,'description']

# afficher le dataframe modifié
data

,title,pubDate,link,description
0,Macron à Pékin : la Chine peut «jouer un rôle ...,"Wed, 05 Apr 2023 16:03:12 +0200",https://www.lefigaro.fr/international/emmanuel...,EN IMAGES - Pour son premier déplacement dans ...
1,"Du général de Gaulle à Emmanuel Macron, retour...","Wed, 05 Apr 2023 11:03:47 +0200",https://www.lefigaro.fr/international/du-gener...,EN IMAGES - Depuis la reconnaissance de la Rép...
2,"À Pékin, Macron veut freiner le rapprochement ...","Wed, 05 Apr 2023 08:45:21 +0200",https://www.lefigaro.fr/international/a-pekin-...,DÉCRYPTAGE - Attendu en Chine pour une visite ...
3,Les confidences des grands patrons français de...,"Mon, 03 Apr 2023 08:31:57 +0200",https://www.lefigaro.fr/societes/les-confidenc...,"ENQUÊTE - Après trois ans d’absence, quinze y ..."
4,"Sommes-nous trop naïfs face à la Chine, comme ...","Wed, 05 Apr 2023 11:25:01 +0200",https://www.lefigaro.fr/international/sommes-n...,
5,Retraites : des dizaines de députés vont écope...,"Wed, 05 Apr 2023 16:36:20 +0200",https://www.lefigaro.fr/politique/retraites-de...,"INFO LE FIGARO - Les parlementaires, dont de n..."
6,Espagne : deux personnes arretées pour trafic ...,"Wed, 05 Apr 2023 16:32:39 +0200",https://www.lefigaro.fr/flash-actu/espagne-deu...,
7,"À Lyon, les maires des grandes villes gérées p...","Wed, 05 Apr 2023 16:31:18 +0200",https://www.lefigaro.fr/lyon/a-lyon-les-maires...,"Les élus Verts d'Annecy, Besançon, Poitiers, T..."
8,Retraites : les syndicats quittent la réunion ...,"Wed, 05 Apr 2023 14:25:13 +0200",https://www.lefigaro.fr/social/retraites-les-s...,"Sans surprise, le rendez-vous de ce matin entr..."
9,"«On a besoin de lui»: Laurent Berger, la mauva...","Wed, 05 Apr 2023 09:38:52 +0200",https://www.lefigaro.fr/politique/on-a-besoin-...,PORTRAIT - Après une série de rendez-vous manq...


In [22]:
# Configure Kafka
bootstrap_servers = ['localhost:9092']
topic = 'your_kafka_topic'
producer = KafkaProducer(bootstrap_servers=bootstrap_servers)

# Configure Cassandra
contact_points = ['localhost']
auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
cluster = Cluster(contact_points=contact_points, auth_provider=auth_provider)
session = cluster.connect()

# Define a sample dataframe
df = data

# Send dataframe to Kafka
serialized_df = df.to_json().encode('utf-8')  # encode the JSON string to bytes
producer.send(topic, value=serialized_df)

# Store dataframe in Cassandra
session.execute("CREATE KEYSPACE IF NOT EXISTS your_keyspace WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1}")
session.execute("USE your_keyspace")
session.execute("CREATE TABLE IF NOT EXISTS RSS (title text, pubdate text,link text,description text, PRIMARY KEY(title))")

for _, row in df.iterrows():
    session.execute(f"INSERT INTO RSS (title, pubdate, link, description ) VALUES ('{row['title']}', '{row['pubDate']}','{row['link']}','{row['description']}')")


SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 1:112 no viable alternative at input 'une' (..., description ) VALUES ('Retraites : des dizaines de députés vont écoper d'[une]...)">

In [31]:
rows = session.execute("SELECT * FROM RSS")
for row in rows:
    print(row.link)

In [34]:
def close_connection():
    producer.flush()
    producer.close()
    session.shutdown()
    cluster.shutdown()
    producer.flush()
    producer.close()
    session.shutdown()
    cluster.shutdown()

In [34]:
def delete_data():
    session.execute("DESCRIBE KEYSPACES;")
    session.execute("USE your_keyspace;")
    session.execute("DESCRIBE TABLES;")
    session.execute("TRUNCATE RSS")

In [35]:
delete_data()